In [1]:
import numpy as np
import os

nframes = []
traj_all = []
date_all = []
glip_feats_all = []
target_filenames = []
phase = 'train'
max_frames = 50

date_dir = "your_path/hand_data_for_pipeline_mask_redcup"
glip_feats_dir = "your_path/glip_feats_redcup"
train_filenames = "your_path/train_split.txt"
test_filenames = "your_path/test_split.txt"

In [2]:
def read_train_test_split(phase, train_filenames, test_filenames):
    if phase == 'train':
        with open(train_filenames, "r") as train_file:
            split_filenames = [line.strip() for line in train_file]
    else:
        with open(test_filenames, "r") as test_file:
            split_filenames = [line.strip() for line in test_file]
    return split_filenames

In [3]:
split_filenames = read_train_test_split(phase, train_filenames, test_filenames)
date_path_all = sorted([p for p in os.listdir(date_dir)])

FileNotFoundError: [Errno 2] No such file or directory: 'your_path/train_split.txt'

In [4]:
# read traj files and vision features
for date_path in date_path_all:
    if not (date_path in split_filenames):
        continue
    target_filenames.append(date_path)
    traj_path = os.path.join(date_dir, date_path, "traj3d_in_cam", "traj3d_in_cam_array.npy")
    traj_data_raw = np.load(traj_path)
    # transpose
    traj_data = np.zeros_like(traj_data_raw)
    traj_data[:, 0] = traj_data_raw[:, 1]
    traj_data[:, 1] = traj_data_raw[:, 0]
    traj_data[: ,2] = traj_data_raw[: ,2]/1000.0 # mm -> m

    traj_all.append(traj_data)
    nframes.append(min(traj_data.shape[0], max_frames))

    glip_feats_path = os.path.join(glip_feats_dir, date_path + ".npy")
    glip_feats = np.load(glip_feats_path)
    glip_feats_all.append(glip_feats)

NameError: name 'date_path_all' is not defined

In [5]:
idx = 0

print(target_filenames[idx])
print(traj_all[idx].shape)
print(glip_feats_all[idx].shape)
print(nframes[idx])

IndexError: list index out of range